# 12 ago 2018

## View 1 - Dados Candidato

```SQL
CREATE VIEW
  eleicoes2014_dados_candidato
AS
  SELECT
    candidatura.candidato_id as id,
    candidato.nome as nome,
    candidatura.nome_urna as nomeUrna,
    candidato.data_nascimento as dataNascimento,
    cidade.nome as cidadeNatal,
    estado.nome as estadoNatal,
    partido.sigla as partido,
    candidatura.numero_candidato as numero,
    candidatura.estado_id as idEstadoCandidatura,
    candidatura.cargo_id as idCargo,
    cargo.nome as cargo,
    grau_instrucao.descricao as grauInstrucao,
    ocupacao.descricao as ocupacao
  FROM
    candidatura
      LEFT JOIN partido on candidatura.partido_id = partido.id
      LEFT JOIN cargo on candidatura.cargo_id = cargo.id
      LEFT JOIN candidato on candidatura.candidato_id = candidato.id
      LEFT JOIN cidade on candidato.cidade_id = cidade.id
      LEFT JOIN estado on cidade.estado_id = estado.id
      LEFT JOIN grau_instrucao on candidato.grau_instrucao_id = grau_instrucao.id
      LEFT JOIN ocupacao on candidato.ocupacao_id = ocupacao.id
  WHERE
    candidatura.eleicao_id = 138 AND
    candidatura.situacao_candidatura_id = 2;
```

```SQL
DROP VIEW eleicoes2014_dados_candidato;
```

## View 2 - Resumo Candidato

```SQL
CREATE
  PROCEDURE build_view_resumo()
  BEGIN
    DECLARE numero_candidatos INT DEFAULT 0;
    DECLARE i INT DEFAULT 0;

    DECLARE id_candidato INT DEFAULT 0;
    DECLARE numero_partidos INT DEFAULT 0;
    DECLARE partidos_anteriores VARCHAR(255) DEFAULT '';
    DECLARE partido_atual VARCHAR(20) DEFAULT '';
    DECLARE numero_processos INT DEFAULT 0;
    DECLARE numero_candidaturas INT DEFAULT 0;
    DECLARE numero_mandatos INT DEFAULT 0;
    DECLARE numero_proposicoes INT DEFAULT 0;
    DECLARE numero_projetos INT DEFAULT 0;

    CREATE TABLE temp (
      id INT,
      numero_partidos INT,
      partidos_anteriores VARCHAR(255),
      partido_atual VARCHAR(20),
      numero_processos INT DEFAULT 0,
      numero_candidaturas INT,
      numero_mandatos INT,
      numero_proposicoes INT,
      numero_projetos INT
    );

    SET numero_candidatos := (SELECT count(*) FROM eleicoes2014_dados_candidato);
    WHILE i < numero_candidatos DO
      SET id_candidato:= (SELECT id FROM eleicoes2014_dados_candidato LIMIT i, 1);

      SET numero_partidos := (
        SELECT
          count(*) as numero_partidos
        FROM
          (SELECT
              partido_id
            FROM
              candidatura
            WHERE
              candidatura.candidato_id = id_candidato
            GROUP BY partido_id) c
      );
      SET partidos_anteriores := (
        SELECT
          GROUP_CONCAT(DISTINCT partido.sigla SEPARATOR ' - ') as partidos_anteriores
        FROM
          candidatura
          LEFT JOIN partido on candidatura.partido_id = partido.id
        WHERE
          candidatura.candidato_id = id_candidato
      );
      SET partido_atual := (
        SELECT
          partido.sigla as partido_atual
        FROM
          candidatura
          LEFT JOIN partido on candidatura.partido_id = partido.id
        WHERE
          candidatura.candidato_id = id_candidato
        ORDER BY
          candidatura.id DESC
        LIMIT 1
      );
      SET numero_processos := 0;
      SET numero_candidaturas := (
        SELECT
          count(DISTINCT candidatura.eleicao_id) as numeroCandidaturas
        FROM
          candidatura
        WHERE
          candidatura.candidato_id = id_candidato
      );
      SET numero_mandatos := (
        SELECT
          count(*) as numero_mandatos
        FROM
          candidatura
        WHERE
          candidatura.candidato_id = id_candidato AND
          (resultado_candidatura_id BETWEEN 1 AND 3 OR resultado_candidatura_id IS NULL)
      );
      SET numero_proposicoes := (
        SELECT
          count(*) as numeroProposicoes
        FROM
          deputado
          INNER JOIN camara_proposicao_deputado cpd on deputado.id = cpd.deputado_id
          INNER JOIN camara_proposicao cp on cpd.camara_proposicao_id = cp.id
        WHERE
          deputado.candidato_id = id_candidato AND
          cp.ano > 2002
      );
      SET numero_projetos := (
        SELECT
          count(*) as numeroProjetos
        FROM
          deputado
          INNER JOIN camara_proposicao_deputado cpd on deputado.id = cpd.deputado_id
          INNER JOIN camara_proposicao cp on cpd.camara_proposicao_id = cp.id
        WHERE
          deputado.candidato_id = id_candidato AND
          cp.tipo_sigla IN ('PEC', 'PFC', 'PL', 'PLP', 'PDC', 'MPV') AND
          cp.ano > 2002
      );
      
      INSERT INTO temp VALUES (
        id_candidato,
        numero_partidos,
        partidos_anteriores,
        partido_atual,
        numero_processos,
        numero_candidaturas,
        numero_mandatos,
        numero_proposicoes,
        numero_projetos
      );
      
      SET i := i + 1;
    END WHILE;
  END;
```